# 範例 : 計程車費率預測
https://www.kaggle.com/c/new-york-city-taxi-fare-prediction

# [作業目標]
- 使用並觀察特徵組合, 在計程車費率預測競賽的影響

# [作業重點]
- 仿造範例並參考今日課程內容, 使用經緯度一圈的長度比的概念造出新特徵, 觀察有什麼影響 (In[6], Out[6])
- 只使用上面所造的這個新特徵, 觀察有什麼影響 (In[7], Out[7])

In [9]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor

df = pd.read_csv('data/taxi_data1.csv')
train_Y = df['fare_amount']
train_x = df.drop(['fare_amount'], axis = 1)
train_x.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2011-10-21 23:54:10 UTC,-73.990580,40.761071,-73.981128,40.758634,2
1,2015-02-03 10:42:03 UTC,-73.988403,40.723431,-73.989647,40.741695,1
2,2014-03-16 18:58:58 UTC,-74.015785,40.715110,-74.012029,40.707888,2
3,2009-06-13 16:10:54 UTC,-73.977322,40.787275,-73.958030,40.778838,3
4,2014-06-12 03:25:56 UTC,-73.989683,40.729717,-73.982490,40.761887,3


In [14]:
train_x['pickup_datetime'] = train_x['pickup_datetime'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S UTC'))


TypeError: strptime() argument 1 must be str, not Timestamp

In [15]:
train_x['year'] = train_x['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%Y'))
train_x['month'] = train_x['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%m'))
train_x['day'] = train_x['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%d'))
train_x['hour'] = train_x['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%H'))
train_x['minute'] = train_x['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%M'))
train_x['second'] = train_x['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%S'))
train_x.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,minute,second
0,2011-10-21 23:54:10,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10
1,2015-02-03 10:42:03,-73.988403,40.723431,-73.989647,40.741695,1,2015,02,03,10,42,03
2,2014-03-16 18:58:58,-74.015785,40.715110,-74.012029,40.707888,2,2014,03,16,18,58,58
3,2009-06-13 16:10:54,-73.977322,40.787275,-73.958030,40.778838,3,2009,06,13,16,10,54
4,2014-06-12 03:25:56,-73.989683,40.729717,-73.982490,40.761887,3,2014,06,12,03,25,56


In [ ]:
train_x = train_x.drop(['pickup_datetime'], axis = 1)

In [20]:
scaler = MinMaxScaler()
train_x1 =scaler.fit_transform(train_x)
Linear = LinearRegression()
print(f'LR score: {cross_val_score(Linear, train_x1, train_Y, cv = 5).mean()}')
GDBT = GradientBoostingRegressor()
print(f'GDBT score: {cross_val_score(GDBT, train_x1, train_Y, cv = 5).mean()}')

C:\Users\Wong Kam Pui\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


LR score: 0.026876871475641616
GDBT score: 0.7114457278043819


In [24]:
train_x['longitude_diff'] = train_x['dropoff_longitude']-train_x['pickup_longitude']
train_x['latitude_diff'] = train_x['dropoff_latitude']-train_x['pickup_latitude']
train_x['distance_2D'] = (train_x['latitude_diff']**2 +train_x['longitude_diff']**2)**0.5
train_x.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,minute,second,longitude_diff,latitude_diff,distance_2D
0,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10,0.009452,-0.002437,0.009761
1,-73.988403,40.723431,-73.989647,40.741695,1,2015,02,03,10,42,03,-0.001244,0.018265,0.018307
2,-74.015785,40.715110,-74.012029,40.707888,2,2014,03,16,18,58,58,0.003756,-0.007222,0.008140
3,-73.977322,40.787275,-73.958030,40.778838,3,2009,06,13,16,10,54,0.019292,-0.008437,0.021056
4,-73.989683,40.729717,-73.982490,40.761887,3,2014,06,12,03,25,56,0.007193,0.032170,0.032964


In [25]:
# 結果 : 準確度上升
train_x2 = scaler.fit_transform(train_x)
print(f'Linear Reg Score : {cross_val_score(Linear, train_x2, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_x2, train_Y, cv=5).mean()}')

C:\Users\Wong Kam Pui\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Linear Reg Score : 0.027479693774541868
Gradient Boosting Reg Score : 0.8047648638326492


# 作業1
* 參考今日教材，試著使用經緯度一圈的長度比這一概念，組合出一個新特徵，再觀察原特徵加上新特徵是否提升了正確率?

In [31]:
import math
"""
Your Code Here, set new character at df['distance_real']
"""
train_x['distance_real'] = ((train_x['latitude_diff'])**2 +(train_x['longitude_diff']*math.cos(40.75))**2)**0.5
# 觀察結果 
train_x3 = scaler.fit_transform(train_x)
print(f'Linear Reg Score : {cross_val_score(Linear, train_x3, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_x3, train_Y, cv=5).mean()}')

C:\Users\Wong Kam Pui\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Linear Reg Score : 0.027630864055463535
Gradient Boosting Reg Score : 0.8050440768112743


In [32]:
train_x.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,minute,second,longitude_diff,latitude_diff,distance_2D,distance_real
0,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10,0.009452,-0.002437,0.009761,0.009723
1,-73.988403,40.723431,-73.989647,40.741695,1,2015,02,03,10,42,03,-0.001244,0.018265,0.018307,0.018307
2,-74.015785,40.715110,-74.012029,40.707888,2,2014,03,16,18,58,58,0.003756,-0.007222,0.008140,0.008133
3,-73.977322,40.787275,-73.958030,40.778838,3,2009,06,13,16,10,54,0.019292,-0.008437,0.021056,0.020984
4,-73.989683,40.729717,-73.982490,40.761887,3,2014,06,12,03,25,56,0.007193,0.032170,0.032964,0.032958


# 作業2
* 試著只使用新特徵估計目標值(忽略原特徵)，效果跟作業1的結果比較起來效果如何?

In [34]:

train_x4 = scaler.fit_transform(train_x[['distance_real']])
print(f'Linear Reg Score : {cross_val_score(Linear, train_x4, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_x, train_Y, cv=5).mean()}')

Linear Reg Score : 0.001428365161551426
Gradient Boosting Reg Score : 0.8045485790283282


*答題2: LINEAR REGRESSION 的分數大跌, GDBT 則相差無幾